In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

# Graphdb configuration
os.environ["NEO4J_URI"]=os.getenv('NEO4J_URI')
os.environ["NEO4J_USERNAME"]=os.getenv('NEO4J_USERNAME')
os.environ["NEO4J_PASSWORD"]=os.getenv('NEO4J_PASSWORD')
os.environ["GROQ_API_KEY"]=os.getenv('GROQ_API_KEY')

In [6]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph()

In [7]:
graph

In [8]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="gemma2-9b-it")

In [5]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7f7400fd0440>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f7400fd0fe0>, model_name='gemma2-9b-it', groq_api_key=SecretStr('**********'))

In [6]:
from langchain_core.documents import Document
text = """
Elon Reeve Musk (born June 28, 1971) is a businessman and investor known for his key roles in space
company SpaceX and automotive company Tesla, Inc. Other involvements include ownership of X Corp.,
formerly Twitter, and his role in the founding of The Boring Company, xAI, Neuralink and OpenAI.
He is one of the wealthiest people in the world; as of July 2024, Forbes estimates his net worth to be
US$221 billion.Musk was born in Pretoria to Maye and engineer Errol Musk, and briefly attended
the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through
his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada.
Musk later transferred to the University of Pennsylvania and received bachelor's degrees in economics
and physics. He moved to California in 1995 to attend Stanford University, but dropped out after
two days and, with his brother Kimbal, co-founded online city guide software company Zip2.
"""
documents = [Document(page_content=text)]

In [7]:
documents

[Document(page_content="\nElon Reeve Musk (born June 28, 1971) is a businessman and investor known for his key roles in space\ncompany SpaceX and automotive company Tesla, Inc. Other involvements include ownership of X Corp.,\nformerly Twitter, and his role in the founding of The Boring Company, xAI, Neuralink and OpenAI.\nHe is one of the wealthiest people in the world; as of July 2024, Forbes estimates his net worth to be\nUS$221 billion.Musk was born in Pretoria to Maye and engineer Errol Musk, and briefly attended\nthe University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through\nhis Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada.\nMusk later transferred to the University of Pennsylvania and received bachelor's degrees in economics\nand physics. He moved to California in 1995 to attend Stanford University, but dropped out after\ntwo days and, with his brother Kimbal, co-founded online city guide sof

In [8]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)

In [9]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [10]:
graph_documents

[GraphDocument(nodes=[Node(id='Elon Reeve Musk', type='Person'), Node(id='Maye', type='Person'), Node(id='Errol Musk', type='Person'), Node(id='Kimbal', type='Person'), Node(id='Spacex', type='Company'), Node(id='Tesla, Inc.', type='Company'), Node(id='X Corp.', type='Company'), Node(id='The Boring Company', type='Company'), Node(id='Xai', type='Company'), Node(id='Neuralink', type='Company'), Node(id='Openai', type='Company'), Node(id='University Of Pretoria', type='University'), Node(id="Queen'S University", type='University'), Node(id='University Of Pennsylvania', type='University'), Node(id='Stanford University', type='University'), Node(id='Zip2', type='Company')], relationships=[Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='Maye', type='Person'), type='PARENT'), Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='Errol Musk', type='Person'), type='PARENT'), Relationship(source=Node(id='Elon Reeve Musk', type='Person'), t

In [11]:
graph_documents[0].nodes

[Node(id='Elon Reeve Musk', type='Person'),
 Node(id='Maye', type='Person'),
 Node(id='Errol Musk', type='Person'),
 Node(id='Kimbal', type='Person'),
 Node(id='Spacex', type='Company'),
 Node(id='Tesla, Inc.', type='Company'),
 Node(id='X Corp.', type='Company'),
 Node(id='The Boring Company', type='Company'),
 Node(id='Xai', type='Company'),
 Node(id='Neuralink', type='Company'),
 Node(id='Openai', type='Company'),
 Node(id='University Of Pretoria', type='University'),
 Node(id="Queen'S University", type='University'),
 Node(id='University Of Pennsylvania', type='University'),
 Node(id='Stanford University', type='University'),
 Node(id='Zip2', type='Company')]

In [12]:
graph_documents[0].relationships

[Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='Maye', type='Person'), type='PARENT'),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='Errol Musk', type='Person'), type='PARENT'),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='Kimbal', type='Person'), type='SIBLING'),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='Spacex', type='Company'), type='FOUNDER'),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='Tesla, Inc.', type='Company'), type='FOUNDER'),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='X Corp.', type='Company'), type='OWNER'),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='The Boring Company', type='Company'), type='FOUNDER'),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='Xai', type='Company'), type='FOUNDER'),
 Relat

In [13]:
## Load the dataset of the movie

movie_query = """
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') |
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') |
    MERGE (g:Genre {name:trim(genre)}) 
    MERGE (m)-[:IN_GENRE]->(g))
"""

In [15]:
graph.query(movie_query)

[]

In [16]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Person {born: INTEGER, name: STRING}
Movie {title: STRING, released: INTEGER, id: STRING, imdbRating: FLOAT}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Person)-[:ACTED_IN]->(:Movie)
(:Person)-[:DIRECTED]->(:Movie)
(:Movie)-[:IN_GENRE]->(:Genre)


In [17]:
from langchain.chains import GraphCypherQAChain
chain = GraphCypherQAChain.from_llm(llm=llm, graph=graph, verbose=True)

In [18]:
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x7f74ac368f20>, cypher_generation_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'schema'], template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7f7400fd0440>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f7400fd0fe0>, model_name='gemma2-9b-it', groq_api_key=SecretStr('**********'))), qa_chain=

In [19]:
response=chain.invoke({"query":"Who was the director of the movie GoldenEye"})
response




> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Movie {title: "GoldenEye"})-[:DIRECTED]->(p:Person)
RETURN p.name

Full Context:
[]

> Finished chain.


{'query': 'Who was the director of the movie GoldenEye',
 'result': "I don't know the answer. \n"}

In [21]:
response=chain.invoke({"query":"tell me the genre of the movie GoldenEye"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "GoldenEye"})-[:IN_GENRE]->(g:Genre) RETURN g.name  

Full Context:
[{'g.name': 'Adventure'}, {'g.name': 'Action'}, {'g.name': 'Thriller'}]

> Finished chain.


{'query': 'tell me the genre of the movie GoldenEye',
 'result': "I don't know the answer. \n"}

In [23]:
print(response['result'])

I don't know the answer. 



In [28]:
response=chain.invoke({"query":"Which actors played in the movie Casino?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "Casino"})-[:ACTED_IN]->(p:Person) RETURN p.name 

Full Context:
[]

> Finished chain.


{'query': 'Which actors played in the movie Casino?',
 'result': "I don't know the answer. \n"}

In [25]:
response=chain.invoke({"query":"Which movie were released in 2008"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie) WHERE m.released = 2008 RETURN m.title 

Full Context:
[{'m.title': 'Ironman'}]

> Finished chain.


{'query': 'Which movie were released in 2008',
 'result': "I don't know the answer. \n"}

In [26]:
response=chain.invoke({"query":"Give me the list of movie having imdb rating more than 8"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie) WHERE m.imdbRating > 8 RETURN m.title 

Full Context:
[{'m.title': 'Toy Story'}, {'m.title': 'Heat'}, {'m.title': 'Casino'}, {'m.title': 'Twelve Monkeys (a.k.a. 12 Monkeys)'}, {'m.title': 'Seven (a.k.a. Se7en)'}, {'m.title': 'Usual Suspects, The'}, {'m.title': 'Hate (Haine, La)'}, {'m.title': 'Braveheart'}, {'m.title': 'Taxi Driver'}, {'m.title': 'Anne Frank Remembered'}]

> Finished chain.


{'query': 'Give me the list of movie having imdb rating more than 8',
 'result': "I don't know the answer.  \n"}

In [27]:
response=chain.invoke({"query":"How many artists are there?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person) RETURN count(p) AS num_artists 

Full Context:
[{'num_artists': 1241}]

> Finished chain.


{'query': 'How many artists are there?',
 'result': 'There are 1241 artists. \n'}

In [ ]:
examples = [
    {
        "question": "How many artists are there?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)",
    },
    {
        "question": "Which actors played in the movie Casino?",
        "query": "MATCH (m:Movie {{title: 'Casino'}})<-[:ACTED_IN]-(a) RETURN a.name",
    },
    {
        "question": "How many movies has Tom Hanks acted in?",
        "query": "MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)",
    },
    {
        "question": "List all the genres of the movie Schindler's List",
        "query": "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name",
    },
    {
        "question": "Which actors have worked in movies from both the comedy and action genres?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name",
    },
    {
        "question": "Which directors have made movies with at least three different actors named 'John'?",
        "query": "MATCH (d:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person) WHERE a.name STARTS WITH 'John' WITH d, COUNT(DISTINCT a) AS JohnsCount WHERE JohnsCount >= 3 RETURN d.name",
    },
    {
        "question": "Identify movies where directors also played a role in the film.",
        "query": "MATCH (p:Person)-[:DIRECTED]->(m:Movie), (p)-[:ACTED_IN]->(m) RETURN m.title, p.name",
    },
    {
        "question": "Find the actor with the highest number of movies in the database.",
        "query": "MATCH (a:Actor)-[:ACTED_IN]->(m:Movie) RETURN a.name, COUNT(m) AS movieCount ORDER BY movieCount DESC LIMIT 1",
    },
]